# Implenting Idempotency using HPE IMC

Try saying that five times fast. 




# What if those VLANS already exist?

There's a concept called idempotency which can be loosely explained as 
>Make sure it's like this.
 If it's not like this, make it like this.
 If it's already like this. Don't do anything
 
Essentially, it's a way to declare the desired configuration state of whatever it is you're trying to configure. If the configuration state of that server, or switch or router is already in that state, than just leave it alone. 

It's a way to ensure that configuration drift doesn't happen. 

So if there's some rabbid network administrator with a console cable running around laughing maniacly as he randomly changes things... 

![jack photo](http://assets.nydailynews.com/polopoly_fs/1.1582165!/img/httpImage/image.jpg_gen/derivatives/gallery_1200/jack-nicholson-shining.jpg)

# Idempotent VLANs

So we're going to look at the last example [here](insert link) where we did the following:
- grabbed the jinja template for vlans directly from a GIThub repository
- grabbed the desired vlans file directly from a GIThub repository
- renderd the Jinja template using the values from the vlan file to get our final config
- used the pyhpeimc library to push the commands through the executecmd RESTful API




## Import Libraries

In [4]:
import requests
import yaml
import time
import githubuser
from pyhpeimc.auth import *
from pyhpeimc.plat.device import *
from pyhpeimc.plat.icc import *
from pyhpeimc.plat.vlanm import *
auth = IMCAuth("http://", "10.101.0.203", "8080", "admin", "admin")

## Download VLANs list from Github

In [5]:
gitauth = githubuser.gitcreds() #you didn't think I was going to give you my password did you?
desired_vlan_list = yaml.load(requests.get('https://raw.githubusercontent.com/netmanchris/Jinja2-Network-Configurations-Scripts/master/vlans.yaml', auth=gitauth).text)


### Print VLANs

In [6]:
print (yaml.dump(desired_vlan_list['vlans'], indent = 4))

- {vlanId: '1', vlanName: default, vlanStatus: '1'}
- {vlanId: '2', vlanName: TenantABC, vlanStatus: '1'}
- {vlanId: '3', vlanName: management, vlanStatus: '1'}
- {vlanId: '6', vlanName: shipping, vlanStatus: '1'}
- {vlanId: '7', vlanName: packing, vlanStatus: '1'}
- {vlanId: '8', vlanName: receiving, vlanStatus: '1'}
- {vlanId: '10', vlanName: mgmt, vlanStatus: '1'}



### Gather just the VLAN IDs

In [7]:
desired_vlan_ids = [vlan['vlanId'] for vlan in desired_vlan_list['vlans']]
desired_vlan_ids

['1', '2', '3', '6', '7', '8', '10']

## Get Current VLANs on Target Device

In [8]:
devid = get_dev_details('10.20.10.10', auth.creds, auth.url)['id']
dev_vlan_list = get_dev_vlans(devid, auth.creds, auth.url)

In [9]:
print (yaml.dump(dev_vlan_list, indent = 4))


- {vlanId: '1', vlanName: default, vlanStatus: '1'}
- {vlanId: '2', vlanName: TenantABC, vlanStatus: '1'}
- {vlanId: '3', vlanName: management, vlanStatus: '1'}
- {vlanId: '5', vlanName: DoesntBelong, vlanStatus: '1'}
- {vlanId: '10', vlanName: mgmt, vlanStatus: '1'}



## Add Desired VLANs to Target Device

In [7]:
help (create_dev_vlan)

Help on function create_dev_vlan in module pyhpeimc.plat.vlanm:

create_dev_vlan(devid, vlanid, vlan_name, auth, url)
    function takes devid and vlanid vlan_name of specific device and 802.1q VLAN tag and issues a RESTFUL call to add the
    specified VLAN from the target device. VLAN Name MUST be valid on target device.
    :param devid: int or str value of the target device
    :param vlanid:int or str value of target 802.1q VLAN
    :param vlan_name: str value of the target 802.1q VLAN name. MUST be valid name on target device.
    :return:HTTP Status code of 201 with no values.



In [10]:
def add_vlans():
    start_time = time.time()
    for vlan in desired_vlan_list['vlans']:
        if vlan['vlanId'] in dev_vlan_ids:
            pass
        else:
            print ('adding vlan ' + str(vlan['vlanId']))
            create_dev_vlan(devid, vlanid=vlan['vlanId'], vlan_name=vlan['vlanName'], auth=auth.creds, url=auth.url)
    print("Operation took --- %s seconds ---" % (time.time() - start_time))
    

In [11]:
dev_vlan_ids = [ vlan['vlanId'] for vlan in (get_dev_vlans(devid, auth.creds, auth.url))]
add_vlans()
get_dev_vlans(devid, auth.creds, auth.url)


adding vlan 6
adding vlan 7
adding vlan 8
Operation took --- 0.5448877811431885 seconds ---


[{'vlanId': '1', 'vlanName': 'default', 'vlanStatus': '1'},
 {'vlanId': '2', 'vlanName': 'TenantABC', 'vlanStatus': '1'},
 {'vlanId': '3', 'vlanName': 'management', 'vlanStatus': '1'},
 {'vlanId': '5', 'vlanName': 'DoesntBelong', 'vlanStatus': '1'},
 {'vlanId': '6', 'vlanName': 'shipping', 'vlanStatus': '1'},
 {'vlanId': '7', 'vlanName': 'packing', 'vlanStatus': '1'},
 {'vlanId': '8', 'vlanName': 'receiving', 'vlanStatus': '1'},
 {'vlanId': '10', 'vlanName': 'mgmt', 'vlanStatus': '1'}]

In [12]:
dev_vlan_ids = [ vlan['vlanId'] for vlan in (get_dev_vlans(devid, auth.creds, auth.url))]
add_vlans()

Operation took --- 2.002716064453125e-05 seconds ---


## Remove Undesired VLANs from Target Device

In [16]:
help (delete_dev_vlans)

Help on function delete_dev_vlans in module pyhpeimc.plat.vlanm:

delete_dev_vlans(devid, vlanid, auth, url)
    function takes devid and vlanid of specific device and 802.1q VLAN tag and issues a RESTFUL call to remove the
    specified VLAN from the target device.
    :param devid: int or str value of the target device
    :param vlanid:
    :return:HTTP Status code of 204 with no values.



In [17]:
def del_vlans():
    start_time = time.time()
    for vlan in get_dev_vlans(devid, auth.creds, auth.url):
        if vlan['vlanId'] not in desired_vlan_ids:
            print ("Deleting vlan " + vlan['vlanId'])
            delete_dev_vlans(devid, vlan['vlanId'], auth.creds, auth.url)
        else:
            print ('Not touching VLAN ' + str(vlan['vlanId']))
    print("Operation took --- %s seconds ---" % (time.time() - start_time))
    

In [19]:
del_vlans()

Not touching VLAN 1
Not touching VLAN 2
Not touching VLAN 3
Not touching VLAN 6
Not touching VLAN 7
Not touching VLAN 8
Not touching VLAN 10
Operation took --- 0.17491388320922852 seconds ---


# Cleaning up After Ourselves

In [20]:
create_dev_vlan(devid, '5', 'DoesntBelong', auth.creds, auth.url)
remove_vlans = [ vlan['vlanId'] for vlan in desired_vlan_list['vlans']]
print (remove_vlans)
for i in remove_vlans:
    delete_dev_vlans(devid, i, auth.creds, auth.url)

['1', '2', '3', '6', '7', '8', '10']
Unable to delete VLAN.
VLAN does not Exist
Device does not support VLAN function
Vlan deleted
Vlan deleted
Vlan deleted
Vlan deleted
Vlan deleted
Vlan deleted


In [21]:
get_dev_vlans(devid, auth.creds, auth.url)

[{'vlanId': '1', 'vlanName': 'default', 'vlanStatus': '1'},
 {'vlanId': '5', 'vlanName': 'DoesntBelong', 'vlanStatus': '1'}]